### Camera

In [ ]:
from jetbot import Camera, bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

camera = Camera()
image = widgets.Image(format='jpeg', width=224, height=224)
#camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

### Task

In [ ]:
import torchvision.transforms as transforms
#from xy_dataset import XYDataset

from dataset import ImageClassificationDataset

TASK = 'object_behavior'

CATEGORIES = ['background1','redlight','greenlight','bottle']

DATASETS = ['A']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
    
#need the dataset.py with the def ImageClassificationDataset
datasets = {}
for name in DATASETS:
    datasets[name] = ImageClassificationDataset(TASK + '_' + name, CATEGORIES, TRANSFORMS)
    
print("{} task with {} categories defined".format(TASK, CATEGORIES))

Initialize jetbot and gamepad controller

In [ ]:
import ipywidgets

controller = widgets.Controller(index=0)
display(controller)

#if no controller set "false"
controller_presence="false"

print("if no controller then just run next cell and make sure controller_presence='false'")

In [ ]:
x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')


button_layout = widgets.Layout(width='128px', height='64px')

x=0 #from here TB
if controller_presence=="true":
    controller = widgets.Controller(index=0)  # replace with index of your controller
    widgets.jsdlink((controller.axes[0], 'value'), (x_slider, 'value'))
    widgets.jsdlink((controller.axes[1], 'value'), (y_slider, 'value'))

button_layout = widgets.Layout(width='200px', height='64px') #TB
free_left = widgets.FloatText(layout=button_layout, value=x, description='forward') #TB
free_right = widgets.FloatText(layout=button_layout, value=x, description='turning') #TB
motorleft = widgets.FloatText(layout=button_layout, value=x, description='Motor Left') #TB
motorright = widgets.FloatText(layout=button_layout, value=x, description='Motor Right') #TB

speed_widget = widgets.FloatSlider(value=0.3, min=0.05, max=1.0, step=0.001, description='speed')
#TB higher speed requires smaller turn_gain values: 2.5 for speed 0.22, around 2 for speed 0.4
turn_gain_widget = widgets.FloatSlider(value=2.5, min=0.05, step=0.001, max=4.0, description='turn sensitivity')
#TB value different for different forward speed, but very small differences
motoradjustment_widget = widgets.FloatSlider(value=0.01, min=0.00, max=0.2, step=0.0001, description='motoradjustment')


In [ ]:
from jetbot import Robot
import math

robot = Robot()

#TB to show the controller values
if controller_presence=="true":
    left_link = traitlets.dlink((controller.axes[1], 'value'), (free_left, 'value'), transform=lambda x:-x)
    right_link = traitlets.dlink((controller.axes[0], 'value'), (free_right, 'value'), transform=lambda x: -x)

def on_value_change(change):
    x= free_right.value
    y= free_left.value
    leftnew, rightnew = steering(x, y)
    motorright.value= round(float(leftnew),3)  
    motorleft.value= round(float(rightnew + motoradjustment_widget.value),3) # adjust the motor that lags behind
    #motoradjustment value important to keep bot driving straight, small offset-values like 0.05
    robot.right_motor.value=motorright.value
    robot.left_motor.value=motorleft.value
        
def steering(x, y): 
    #script from stackexchange of user Pedro Werneck 
    #https://electronics.stackexchange.com/questions/19669/algorithm-for-mixing-2-axis-analog-input-to-control-a-differential-motor-drive
    # convert to polar
    global controller_presence
    if controller_presence=="true":
        r = math.hypot(x, y)
        t = math.atan2(y, x)
    

        # rotate by 45 degrees
        t += math.pi / -4.0

        # back to cartesian
        left = r * math.cos(t)
        right = r * math.sin(t)

        # rescale the new coords
        left = left * math.sqrt(2)
        right = right * math.sqrt(2)

        # clamp to -1/+1
        scalefactor= speed_widget.value
        left = max(scalefactor*-1.0, min(left, scalefactor))
        right = max(scalefactor*-1.0, min(right, scalefactor))
    
        #gamma correction for response sensitivity of joystick while turning : TB
        gamma=turn_gain_widget.value  #using slider for joystick 1-4, for object recognition 2-40 
        if left <0 :
            left= -1* (((abs(left)/scalefactor)**(1/gamma))*scalefactor)
        else:
            left= ((abs(left)/scalefactor)**(1/gamma))*scalefactor
               
        if right <0:
            right= -1*(((abs(right)/scalefactor)**(1/gamma))*scalefactor)
        else:
            right= ((abs(right)/scalefactor)**(1/gamma))*scalefactor
    
    return left, right


free_left.observe(on_value_change, names='value')
free_right.observe(on_value_change, names='value')

#left_link = traitlets.dlink((motorleft, 'value'), (robot.left_motor, 'value'))
#right_link = traitlets.dlink((motorright, 'value'), (robot.right_motor, 'value'))

### Data Collection

In [ ]:
# initialize active dataset
dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera in case we are running this cell for second time
#camera.unobserve_all()

# create image preview
camera_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')
save_widget = ipywidgets.Button(description='add')


# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)


# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

# save image for category and update counts
def save(c):
    dataset.save_entry(camera.value, category_widget.value)
    count_widget.value = dataset.get_count(category_widget.value)
save_widget.on_click(save)

        


data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget]), dataset_widget, category_widget, count_widget, save_widget
])
'''
data_collection_widget = ipywidgets.VBox([
    dataset_widget, category_widget, count_widget, save_widget
])
'''

# display(data_collection_widget)
print("data_collection_widget created")

### Model

In [ ]:
import torch
import torchvision

device = torch.device('cuda')
output_dim = len(dataset.categories)  #number of categories


# ALEXNET
# model = torchvision.models.alexnet(pretrained=True)
# model.classifier[-1] = torch.nn.Linear(4096, output_dim)

# SQUEEZENET 
# model = torchvision.models.squeezenet1_1(pretrained=True)
# model.classifier[1] = torch.nn.Conv2d(512, output_dim, kernel_size=1)
# model.num_classes = len(dataset.categories)

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)


# RESNET 34
# model = torchvision.models.resnet34(pretrained=True)
# model.fc = torch.nn.Linear(512, output_dim)

# DENSENET 121
# model = torchvision.models.densenet121(pretrained=True)
# model.classifier = torch.nn.Linear(model.num_features, output_dim)

model = model.to(device).eval()
# thanks 2 abuelgasimsaadeldin 
model_save_button = ipywidgets.Button(description='save model')
model_load_button = ipywidgets.Button(description='load model')
model_path_widget = ipywidgets.Text(description='model path', value='classification_behavior_model.pth')

def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))
model_load_button.on_click(load_model)
    
def save_model(c):
    torch.save(model.state_dict(), model_path_widget.value)
model_save_button.on_click(save_model)

model_widget = ipywidgets.VBox([
    model_path_widget,
    ipywidgets.HBox([model_load_button, model_save_button])
])

print("model configured and model_widget created")
#display(model_widget)

### Live Execution

In [ ]:
import threading
import time
from utils import preprocess
import torch.nn.functional as F

state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
prediction_widget = ipywidgets.Text(description='prediction')
score_widgets = []
for category in dataset.categories:
    score_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, description=category, orientation='vertical')
    score_widgets.append(score_widget)

def live(state_widget, model, camera, prediction_widget, score_widget):
    global dataset
    while state_widget.value == 'live':
        image = camera.value
        preprocessed = preprocess(image)
        output = model(preprocessed)
        output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
        category_index = output.argmax()
        prediction_widget.value = dataset.categories[category_index]
        for i, score in enumerate(list(output)):
            score_widgets[i].value = score
            
def start_live(change):
    if change['new'] == 'live':
        execute_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget, score_widget))
        execute_thread.start()

state_widget.observe(start_live, names='value')

live_execution_widget = ipywidgets.VBox([
    ipywidgets.HBox(score_widgets),
    prediction_widget,
    state_widget
])

# display(live_execution_widget)
print("live_execution_widget")

In [ ]:
BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

epochs_widget = ipywidgets.IntText(description='epochs', value=1)
eval_button = ipywidgets.Button(description='evaluate')
train_button = ipywidgets.Button(description='train')
loss_widget = ipywidgets.FloatText(description='loss')
accuracy_widget = ipywidgets.FloatText(description='accuracy')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')

def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget
    
    try:
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        state_widget.value = 'stop'
        train_button.disabled = True
        eval_button.disabled = True
        time.sleep(1)

        if is_training:
            model = model.train()
        else:
            model = model.eval()

        while epochs_widget.value > 0:
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            for images, labels in iter(train_loader):
                # send data to device
                images = images.to(device)
                labels = labels.to(device)

                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()

                # execute model to get outputs
                outputs = model(images)

                # compute loss
                loss = F.cross_entropy(outputs, labels)

                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()

                # increment progress
                error_count += len(torch.nonzero(outputs.argmax(1) - labels).flatten())
                count = len(labels.flatten())
                i += count
                sum_loss += float(loss)
                progress_widget.value = i / len(dataset)
                loss_widget.value = sum_loss / i
                accuracy_widget.value = 1.0 - error_count / i
                
            if is_training:
                epochs_widget.value = epochs_widget.value - 1
            else:
                break
    except e:
        pass
    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False
    state_widget.value = 'live'
    
train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))
    
train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    accuracy_widget,
    ipywidgets.HBox([train_button, eval_button])
])

#display(train_eval_widget)
print("trainer configured and train_eval_widget created")

### All together!

The following widget can be used to label a multi-class x, y dataset.  It supports labeling only one instance of each class per image (ie: only one dog), but multiple classes (ie: dog, cat, horse) per image are possible.

Click the image on the top left to save an image of ``category`` to ``dataset`` at the clicked location.

| Widget | Description |
|--------|-------------|
| dataset | Selects the active dataset |
| category | Selects the active category |
| epochs | Sets the number of epochs to train for |
| train | Trains on the active dataset for the number of epochs specified |
| evaluate | Evaluates the accuracy on the active dataset over one epoch |
| model path | Sets the active model path |
| load | Loads a model from the active model path |
| save | Saves a model to the active model path |
| stop | Disables the live demo |
| live | Enables the live demo |

In [ ]:
# Combine all the widgets into one display
all_widget = ipywidgets.VBox([
    ipywidgets.HBox([data_collection_widget, live_execution_widget]), 
    train_eval_widget,
    model_widget
])

display(all_widget)

In [ ]:
#if no controller then do not run this cell
display(controller)

display(widgets.VBox([
    speed_widget,
    turn_gain_widget,
    motoradjustment_widget,
]))

display(widgets.HBox([free_left, free_right, motorleft, motorright]))

before stopping kernel execute the last cell, camera settings are in Jetcam

In [ ]:
#for end of taks and also for training of many images disconnect camera and stop bot
import time
camera.stop()
time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
robot.stop()